# Cluster map

In [1]:
%matplotlib inline

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import pyarrow

In [3]:
# Verify SparkContext
print(sc)

# Print Spark version
print(sc.version)

<SparkContext master=mesos://vm-75063.lal.in2p3.fr:5050 appName=PySparkShell>
2.4.4


In [4]:
from pyspark.sql.functions import col, pandas_udf 
from pyspark.sql.types import LongType

from pyspark.sql.types import IntegerType,FloatType
from pyspark.sql.functions import pandas_udf, PandasUDFType

from pyspark.sql import functions as F
from pyspark.sql.functions import randn


In [5]:
# here is how we create a function ('Ang2Pix') that can be called by dataframes
# it takes as input the "ra" and "dec" values (which are not very different from theta/phi)
# and returns the pixel number (but as pandas series for efficiency)
import numpy as np
import pandas as pd
import healpy as hp

nside=512

In [6]:
! hadoop dfs -ls /lsst/DC2/cosmoDC2

DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

Found 5 items
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-09-03 20:04 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-10-16 10:52 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_double.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-04-23 17:28 /lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image_nofaint.parquet
drwxr-xr-x   - stephane.plaszczynski lsst          0 2020-09-07 19:17 /lsst/DC2/cosmoDC2/shear_cosmoDC2
drwxr-xr-x   - stephane.plaszczynski lsst          0 2019-04-16 18:40 /lsst/DC2/cosmoDC2/xyz_v1.1.4_hive


In [7]:
base_dir="/lsst/DC2/cosmoDC2/shear_cosmoDC2"

In [8]:
fullfilename="/lsst/DC2/cosmoDC2/cosmoDC2_v1.1.4_image.parquet"
datafile=fullfilename
FLAG_ONE_FILE_INPUT=True

In [9]:
#from subprocess import Popen, PIPE
#hdfs_path = base_dir
#process = Popen(f'hdfs dfs -ls -h {hdfs_path}', shell=True, stdout=PIPE, stderr=PIPE)
#std_out, std_err = process.communicate()
#list_of_file_names = [fn.split(' ')[-1].split('/')[-1] for fn in std_out.decode().readlines()[1:]][:-1]
#list_of_file_names_with_full_address = [fn.split(' ')[-1] for fn in std_out.decode().readlines()[1:]][:-1]

In [10]:
#for fn in std_out.readlines():
#    print(fn)

In [11]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

In [12]:
# Read the data as DataFrame
if FLAG_ONE_FILE_INPUT:
    df = spark.read.format("parquet").load(datafile)
else: # read all files
    #the_list_of_files=os.listdir(base_dir)
    #the_sorted_list_of_files=sorted(the_list_of_files)
    #print("sorted files : ",the_sorted_list_of_files[:5])
    #selected_files=[]
    #for filename in the_sorted_list_of_files:
    #    sel_filename=re.findall("^dc2_object_run2.2i_dr6c_tract[0-9][0-9][0-9][0-9].parquet$",filename)
    #    if len(sel_filename) > 0:
    #        selected_files.append(sel_filename[0])
    #print("selected files : ", selected_files[:5])
    #InputPath = [base_dir + "/{}".format(filename) for filename in selected_files]
    InputPath = all_files  # filename from the decoded index file
    df = spark.read.parquet(*InputPath)        

In [13]:
df.printSchema()

root
 |-- Mag_true_g_lsst_z0: float (nullable = true)
 |-- Mag_true_z_lsst_z0: float (nullable = true)
 |-- ra: float (nullable = true)
 |-- mag_y: float (nullable = true)
 |-- mag_z: float (nullable = true)
 |-- size_true: float (nullable = true)
 |-- halo_id: long (nullable = true)
 |-- position_x: float (nullable = true)
 |-- mag_g: float (nullable = true)
 |-- Mag_true_u_lsst_z0: float (nullable = true)
 |-- mag_true_r: float (nullable = true)
 |-- position_angle_true: float (nullable = true)
 |-- mag_true_g: float (nullable = true)
 |-- stellar_mass: float (nullable = true)
 |-- Mag_true_y_lsst_z0: float (nullable = true)
 |-- position_y: float (nullable = true)
 |-- mag_true_i: float (nullable = true)
 |-- redshift: float (nullable = true)
 |-- Mag_true_i_lsst_z0: float (nullable = true)
 |-- mag_r: float (nullable = true)
 |-- mag_true_y: float (nullable = true)
 |-- dec: float (nullable = true)
 |-- mag_true_u: float (nullable = true)
 |-- Mag_true_r_lsst_z0: float (nullable = 

In [14]:
data=df.select("halo_id","ra","dec","position_x","position_y","position_z","redshift","stellar_mass","is_central")

In [15]:
data.show()

+-----------+---------+----------+----------+----------+----------+------------+-------------+----------+
|    halo_id|       ra|       dec|position_x|position_y|position_z|    redshift| stellar_mass|is_central|
+-----------+---------+----------+----------+----------+----------+------------+-------------+----------+
|     151487|49.312595| -40.29579| 45.516796|  52.94169|-59.201447|  0.02164939|1.61537874E10|      true|
|     151487|49.477528|-40.279537| 45.421547| 53.139584|-59.242195|  0.02191215| 1.45056048E8|     false|
|  100151487|51.234653|-40.120605| 43.541767|  54.22216|-58.601585| 0.021491187| 3.39115392E8|      true|
|  200151487| 49.46697|-39.989403| 45.164555| 52.819206|-58.292233| 0.021335145| 2.54158784E8|      true|
|  300151487|50.803635|-40.039272| 12.911323| 15.832897| -17.16664| 0.006085375| 2.73879296E9|      true|
|  400151487| 50.54519| -40.07799|  13.13755| 15.962756|-17.395363|0.0062329015|  4.0522228E7|      true|
|  500151487|51.027073|-40.140404| 45.259895| 

In [16]:
data.count()

2256249331

In [17]:
#data.describe().show()

In [18]:
center_clu=data.filter(data.is_central==True)

In [19]:
center_clu.count()

2173500274

In [20]:
center_clu.show()

+-----------+---------+----------+----------+----------+----------+------------+-------------+----------+
|    halo_id|       ra|       dec|position_x|position_y|position_z|    redshift| stellar_mass|is_central|
+-----------+---------+----------+----------+----------+----------+------------+-------------+----------+
|     151487|49.312595| -40.29579| 45.516796|  52.94169|-59.201447|  0.02164939|1.61537874E10|      true|
|  100151487|51.234653|-40.120605| 43.541767|  54.22216|-58.601585| 0.021491187| 3.39115392E8|      true|
|  200151487| 49.46697|-39.989403| 45.164555| 52.819206|-58.292233| 0.021335145| 2.54158784E8|      true|
|  300151487|50.803635|-40.039272| 12.911323| 15.832897| -17.16664| 0.006085375| 2.73879296E9|      true|
|  400151487| 50.54519| -40.07799|  13.13755| 15.962756|-17.395363|0.0062329015|  4.0522228E7|      true|
|  500151487|51.027073|-40.140404| 45.259895| 55.945347|  -60.6833| 0.022323765| 2.88517056E8|      true|
|    -151487| 50.63813|-39.092896|  27.74802| 

In [ ]:
center_clu=center_clu.withColumn("logmass", gal["Z"]+F.randn()*(0.02*(1+gal["Z"])))

In [21]:
from pyspark.ml.feature import Bucketizer

In [22]:
minmax = center_clu.agg(F.min("stellar_mass"), F.max("stellar_mass")).head()

In [23]:
minmax

Row(min(stellar_mass)=3536.082275390625, max(stellar_mass)=4511717916672.0)

In [ ]:
np.